In [1]:
!pip install "unsloth[colab-new]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 29.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 70.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.6/273.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 14.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.5/155.5 MB 9.3 MB/s eta 0:00:00:00:0100:

In [2]:
%%writefile generate_submission.py
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from transformers import AutoTokenizer
from tqdm import tqdm
import pandas as pd
import os
from peft import PeftModel

# --- 1. CONFIG ---
# --- THIS IS THE CORRECT PATH ---
MODEL_PATH = "/kaggle/input/trained-deep-learning/other/default/1" 

MAX_SEQ_LENGTH = 4096
OUTPUT_FILENAME = "submission.csv"

print(f"--- GENERATING SUBMISSION (v4) ---")
print(f"Loading base model...")

# --- 2. LOAD MODEL (Corrected 2-Step Process) ---
# Step 2a: Load the base Llama 3 model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = None,
    load_in_4bit = True,
    device_map = "auto",
)

# Step 2b: Load your saved adapters (the PEFT model) onto the base model
print(f"Attaching  adapters from: {MODEL_PATH}")
model = PeftModel.from_pretrained(model,MODEL_PATH)
print("Adapters loaded successfully.")


FastLanguageModel.for_inference(model)

# --- 3. LOAD TEST DATA ---
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")

# --- 4. DEFINE INFERENCE PROMPT & PARSER ---
inference_prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a meticulous math solution verifier. Your task is to carefully analyze the 'Provided Solution' step-by-step to determine if it logically and correctly reaches the 'Expected Answer' for the given 'Question'. Respond ONLY with 'True' if the solution's reasoning AND final result are correct and match the expected answer, otherwise respond ONLY with 'False'.<|eot_id|><|start_header_id|>user<|end_header_id|>
Question:
{}

Expected Answer:
{}

Provided Solution:
{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

def parse_output(response_text):
    try:
        output_part = response_text.split("<|start_header_id|>assistant<|end_header_id|>")[1]
    except IndexError:
        return False
    output_part = output_part.replace("<|eot_id|>", "").strip()
    return output_part.lower().startswith('true')

# --- 5. RUN INFERENCE LOOP ---
print(f"Running inference on {len(test_dataset)} test samples...")
predictions = []
for example in tqdm(test_dataset):
    prompt = inference_prompt_template.format(
        example["question"],
        str(example["answer"]),
        str(example["solution"])
    )
    
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=8,
        eos_token_id=terminators,
        pad_token_id=tokenizer.pad_token_id,
        use_cache=True
    )
    
    response_text = tokenizer.batch_decode(outputs)[0]
    prediction = parse_output(response_text)
    predictions.append(prediction)

# --- 6. SAVE SUBMISSION.CSV ---
print(f"Creating {OUTPUT_FILENAME}...")
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})
submission.to_csv(OUTPUT_FILENAME, index=False)

print(f"\n--- ✅ Test submission file '{OUTPUT_FILENAME}' is ready! ---")
print(submission.head())

Writing generate_submission.py


In [3]:
!python generate_submission.py

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
2025-11-02 21:22:59.294129: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762118579.710051     114 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762118579.826871     114 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🦥 Unsloth Zoo will now patch everything to make training faster!
--- GENERATING SUBMISSION (v4) ---
Loading base model...
==((====))==  Unsloth 2025.10.12: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.4.0
\       